# Find best Features:
- we create combination of all the features which are not encoded by baseNencoder and try all of them and see if we might get better result

## Import Libraries and funtions

In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
!pip install category_encoders

     |████████████████████████████████| 80 kB 5.8 MB/s 


In [3]:
# libraries
from sklearn.model_selection import train_test_split , cross_val_score

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier 
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier



import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statistics
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from sklearn.preprocessing import MinMaxScaler
from category_encoders.basen import BaseNEncoder
from category_encoders.count import CountEncoder
import itertools
from sklearn.metrics import accuracy_score

In [4]:
data_all_filtered= pd.read_csv('data/data_filtered.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
data_all_filtered.drop(columns=['Cabin_location'],inplace=True)
data_all_filtered.shape, train.shape, test.shape 

((1309, 13), (891, 12), (418, 11))

In [5]:
def split_trainANDtest(data_all_filtered):
  data_train = data_all_filtered.iloc[0:train.shape[0],:]
  data_test = data_all_filtered.iloc[train.shape[0]:,:]
  return data_train, data_test

def all_combinations(categorical_columns):
  all_combinations = []
  for r in range(len(categorical_columns) + 1):
    combinations_object = itertools.combinations(categorical_columns, r)
    combinations_list = list(combinations_object)
    all_combinations += combinations_list
  all_combinations = [list(x) for x in all_combinations][1:]
  return all_combinations

def Min_Max_Scaler_X(train_enc, val_enc):
  scaler = MinMaxScaler()
  train_enc = scaler.fit_transform(train_enc)
  val_enc = scaler.transform(val_enc)
  return train_enc, val_enc

def train_clasifier(classifier_X,X_train,y_train,X_val,y_val):
  baseline = classifier_X
  scores_cv = cross_val_score(baseline, X_train, y_train, cv=5,scoring='accuracy')
  model = baseline.fit(X_train,y_train)
  scores_pred = accuracy_score(y_val,model.predict(X_val))
  return scores_cv, scores_pred


classifiers_list = [
      XGBClassifier(random_state=42), 
      RandomForestClassifier(random_state=42), 
      LGBMClassifier(random_state=42),   
      KNeighborsClassifier(), 
      SVC()
                          ]

#import from preous notebook
label_encoder_cols =['Fare_bin', 'Embarked']
counter_encoder_cols = ['Family_size', 'Title', 'age_bin']
print(label_encoder_cols, counter_encoder_cols)

['Fare_bin', 'Embarked'] ['Family_size', 'Title', 'age_bin']


In [6]:
data_train, data_test = split_trainANDtest(data_all_filtered)
data_train.shape, data_test.shape
#test
X_train, X_val, y_train, y_val = train_test_split(data_train.drop(columns=['Survived']),data_train['Survived'] , test_size=0.2, random_state=42, shuffle=True)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

X_train['Fare_bin'] = X_train['Fare_bin'].map({'cheap':0,'medium':1,'high':2,'very_high':3})
X_train['Embarked'] = X_train['Embarked'].map({'S':0,'C':1,'Q':2})

X_val['Fare_bin'] = X_val['Fare_bin'].map({'cheap':0,'medium':1,'high':2,'very_high':3})
X_val['Embarked'] = X_val['Embarked'].map({'S':0,'C':1,'Q':2})

best_encoder = CountEncoder(cols=counter_encoder_cols,return_df=True, handle_unknown=0)
X_train = best_encoder.fit_transform(X_train,y_train)
X_val = best_encoder.transform(X_val)

(712, 12) (712,) (179, 12) (179,)


In [7]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,is_Alone,Embarked,Fare_bin,Family_size,Title,age_bin
331,1,0,46.0,0,0,28.5,1,0,1,429,419,502
733,2,0,23.0,0,0,13.0,1,0,1,429,419,502


In [8]:
X_val.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,is_Alone,Embarked,Fare_bin,Family_size,Title,age_bin
709,3,0,17.0,1,1,15.2458,0,1,1,206,33,104
439,2,0,31.0,0,0,10.5000,1,0,1,429,419,502


In [9]:
all_columns = X_val.columns.tolist()
# encoded_cols = list(set(X_val.columns) - set(other_columns_exapt_encoded))

In [10]:
all_column_comb = all_combinations(all_columns)
all_column_comb = [x for x in all_column_comb if len(x)>=9]
len(all_column_comb)

299

In [11]:
list_classifiers = [XGBClassifier(random_state=42), RandomForestClassifier(random_state=42), 
                    LGBMClassifier(random_state=42), SVC(random_state=42) , KNeighborsClassifier()]
# other_columns_exapt_encoded = ['Pclass','Sex','Age','SibSp','Parch','Fare','is_Alone','Fare_bin','Family_size','age_bin']
# encoded_cols = list(set(X_val.columns) - set(other_columns_exapt_encoded))

In [12]:
import gc
statistic = []
score = 0
for j, model in enumerate(list_classifiers):
  print("***********************************************************")
  max_score = 0
  best_features = 'None'
  for i,features in enumerate(all_column_comb):
        selected_features = features
        #select columns
        selected_X_train = X_train[selected_features]
        selected_X_val = X_val[selected_features]
        
        #min_max_scaler
        selected_X_train_scaled, selected_X_val = Min_Max_Scaler_X(selected_X_train,selected_X_val)
        
        # calculate score
        score_CV, pred_score = train_clasifier(model, selected_X_train_scaled, y_train,selected_X_val,y_val)

        #scores
        mean_score_cv = round(score_CV.mean(),4)
        std_score_cv = round(score_CV.std(), 4)
        # our goal is to maximize (mean_score_cv+pred_score) and minimize std_score_cv
        total_score = round((mean_score_cv+pred_score)/2,4)
        print(i, " |  " , total_score, "|   ", mean_score_cv, " |", std_score_cv, "|  ",  pred_score ,"|  ", features )
        gc.collect()
        #model,  mean_cv_score,  std_cv_score,  pred_score,  features
        statistic.append((j,i,total_score, mean_score_cv,std_score_cv, pred_score, features))

***********************************************************
0  |   0.8235 |    0.8314  | 0.0104 |   0.8156424581005587 |   ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'is_Alone', 'Embarked', 'Fare_bin']
1  |   0.827 |    0.8328  | 0.0141 |   0.8212290502793296 |   ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'is_Alone', 'Embarked', 'Family_size']
2  |   0.8326 |    0.8272  | 0.0313 |   0.8379888268156425 |   ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'is_Alone', 'Embarked', 'Title']
3  |   0.8256 |    0.83  | 0.0154 |   0.8212290502793296 |   ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'is_Alone', 'Embarked', 'age_bin']
4  |   0.8144 |    0.83  | 0.0227 |   0.7988826815642458 |   ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'is_Alone', 'Fare_bin', 'Family_size']
5  |   0.8333 |    0.8342  | 0.0282 |   0.8324022346368715 |   ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'is_Alone', 'Fare_bin', 'Title']
6  |   0.8228 |    0.83  | 0.0154 |   0.815642

**our goal**: for each classifier, find feature list which has:
- maximum total score (mean score +pred)/2
- minimum std

we can do it by doing averaging pred and mean score


In [13]:
len(statistic)

1495

In [15]:
for i in range(len(list_classifiers)):
  print(i,": ", [ x for x in statistic if x[0]==i if x[2]== max([x[2] for x in statistic if x[0]==i]) ])

0 :  [(0, 62, 0.8438, 0.8328, 0.0399, 0.8547486033519553, ['Pclass', 'Sex', 'Age', 'Parch', 'Fare', 'is_Alone', 'Fare_bin', 'Family_size', 'Title'])]
1 :  [(1, 54, 0.8332, 0.8061, 0.0164, 0.8603351955307262, ['Pclass', 'Sex', 'Age', 'SibSp', 'is_Alone', 'Fare_bin', 'Family_size', 'Title', 'age_bin'])]
2 :  [(2, 77, 0.8402, 0.8089, 0.0209, 0.8715083798882681, ['Pclass', 'Sex', 'Age', 'Fare', 'is_Alone', 'Embarked', 'Fare_bin', 'Family_size', 'Title'])]
3 :  [(3, 92, 0.8256, 0.8244, 0.0225, 0.8268156424581006, ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'is_Alone', 'Fare_bin', 'Title', 'age_bin']), (3, 286, 0.8256, 0.8244, 0.0271, 0.8268156424581006, ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'is_Alone', 'Embarked', 'Fare_bin', 'Family_size', 'Title'])]
4 :  [(4, 18, 0.841, 0.8216, 0.0338, 0.8603351955307262, ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_bin', 'Title', 'age_bin'])]


**Result**:
1. Classifier0:    ['Pclass', 'Sex', 'Age', 'Parch', 'Fare', 'is_Alone', 'Fare_bin', 'Family_size', 'Title']
2. Classifier1:   ['Pclass', 'Sex', 'Age', 'SibSp', 'is_Alone', 'Fare_bin', 'Family_size', 'Title', 'age_bin']
3. Classifier2:     ['Pclass', 'Sex', 'Age', 'Fare', 'is_Alone', 'Embarked', 'Fare_bin', 'Family_size', 'Title']
4. Classifier3:     ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'is_Alone', 'Embarked', 'Fare_bin', 'Family_size', 'Title']
5. Classifier4:   ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_bin', 'Title', 'age_bin']